In [10]:
# count_food2.py

from collections import Counter

import spacy
from spacy.pipeline import EntityRuler
import pandas as pd

# 1. 口コミデータの読み込み
df = pd.read_csv('text/miyajima_sample.csv', encoding='utf-8',
                 parse_dates=['旅行の時期'])

nlp = spacy.load('ja_ginza')

In [11]:
# 2. 固有表現抽出のルールを追加
patterns = [
    {'label': 'Dish', 'pattern': [
        {'TEXT': 'あなご'},
        {'TEXT': {'REGEX': r'飯|めし'}},
    ]},
    {'label': 'Dish', 'pattern': '揚げもみじ'},
    {'label': 'Dish', 'pattern': 'ガンス'},
    {'label': 'Dish', 'pattern': [
        {'TEXT': 'にぎり'},
        {'TEXT': '天'}
    ]}
]

ruler = EntityRuler(nlp, overwrite_ents=True)
ruler.add_patterns(patterns)
nlp.add_pipe(ruler,  before='ner')

In [12]:
# 3. 料理名をリストに追加
food = []
for doc in nlp.pipe(df['口コミ']):
    # 料理名（Dishタグ）を抽出
    for ent in doc.ents:
        if ent.label_ == 'Dish':
            food.append(ent.text)

# 4. 料理名の頻度をカウント
counter = Counter(food)

In [13]:
# 5. 表記ゆれを解消するための用語辞書を用意
# 用語辞書のキーは代表語，値は表記ゆれの単語のリスト
name2variants = {'あなご飯': ['穴子飯', 'あなごめし', '穴子めし'],
                 '揚げもみじ': ['揚げ紅葉', '揚げ紅葉饅頭']}

# 用語辞書を，表記揺れをチェックしやすい形式に変換
variant2name = {}       # キーは表記ゆれの単語，値は代表語
for name, variants in name2variants.items():
    variant2name[name] = name
    for variant in variants:
        variant2name[variant] = name

In [14]:
# 6. 表記ゆれを解消して料理名の頻度をカウント
counter2 = Counter()
for word, count in counter.items():
    name = variant2name.get(word, word)
    counter2[name] += count

# 7. 料理名の頻度を表示
print('count word')
for word, count in counter2.most_common():
    print(f'{count:>5} {word}')

count word
   31 もみじ饅頭
   29 揚げもみじ
   15 あなご飯
   11 焼き牡蠣
    8 饅頭
    6 にぎり天
    6 ガンス
    5 コーヒー
    4 牡蠣料理
    4 アイスクリーム
    4 お茶
    3 鹿せんべい
    3 穴子
    3 タピオカ
    3 クリーム
    2 サンドdeおにぎり
    2 アナゴ飯
    2 宮島ビール
    2 チーズ
    2 ビール
    2 日本酒
    1 昼食
    1 カクテル
    1 ジュース
    1 おにぎり
    1 勝谷菓子パン舗
    1 アナゴ飯弁当
    1 地ビール
    1 サイコロステーキ
    1 ケーキ
    1 作り立て
    1 アイス
    1 せんべい
    1 穴子入り竹輪
    1 スターバックス
    1 レモンジュース
    1 抹茶
    1 包々浦自然公園
    1 お好み焼き
    1 自然公園
    1 多宝塔
    1 包々浦
    1 カワウソ
    1 殻付き牡蠣
    1 牡蠣のパスタ
    1 牡蠣入り
    1 海鮮丼
    1 牡蠣入りのカレーパン
    1 ごはん
    1 焼き立て
    1 スターバックスコーヒー
    1 プリップリ
    1 七味
    1 かきフライ
    1 醤油マヨ
    1 もみじ饅頭焼き


In [15]:
# 「男性」の口コミに出現する料理名の頻度をカウント

# 料理名をリストに追加
food = []
for doc in nlp.pipe(df[df['性別'] == '男性']['口コミ']):  # 女性の口コミに限定
    # 料理名（Dishタグ）を抽出
    for ent in doc.ents:
        if ent.label_ == 'Dish':
            food.append(ent.text)

# 料理名の頻度をカウント
counter = Counter(food)

# 表記ゆれを解消して料理名の頻度をカウント
counter2 = Counter()
for word, count in counter.items():
    name = variant2name.get(word, word)
    counter2[name] += count

# 料理名の頻度を表示
print('●　男性の場合')
print('count word')
for word, count in counter2.most_common():
    print(f'{count:>5} {word}')

●　男性の場合
count word
   22 もみじ饅頭
   13 揚げもみじ
    9 あなご飯
    8 焼き牡蠣
    5 饅頭
    3 鹿せんべい
    3 お茶
    2 チーズ
    2 クリーム
    2 穴子
    2 牡蠣料理
    2 日本酒
    1 コーヒー
    1 サイコロステーキ
    1 ケーキ
    1 せんべい
    1 穴子入り竹輪
    1 スターバックス
    1 包々浦自然公園
    1 アナゴ飯
    1 お好み焼き
    1 自然公園
    1 多宝塔
    1 包々浦
    1 海鮮丼
    1 にぎり天
    1 プリップリ
    1 七味
    1 かきフライ
    1 ビール
    1 宮島ビール
    1 醤油マヨ


In [16]:
# 「女性」の口コミに出現する料理名の頻度をカウント

# 料理名をリストに追加
food = []
for doc in nlp.pipe(df[df['性別'] == '女性']['口コミ']):  # 女性の口コミに限定
    # 料理名（Dishタグ）を抽出
    for ent in doc.ents:
        if ent.label_ == 'Dish':
            food.append(ent.text)

# 料理名の頻度をカウント
counter = Counter(food)

# 表記ゆれを解消して料理名の頻度をカウント
counter2 = Counter()
for word, count in counter.items():
    name = variant2name.get(word, word)
    counter2[name] += count

# 料理名の頻度を表示
print('●　女性の場合')
print('count word')
for word, count in counter2.most_common():
    print(f'{count:>5} {word}')

●　女性の場合
count word
   16 揚げもみじ
    9 もみじ饅頭
    6 あなご飯
    6 ガンス
    5 にぎり天
    4 アイスクリーム
    4 コーヒー
    3 焼き牡蠣
    3 饅頭
    3 タピオカ
    2 牡蠣料理
    2 サンドdeおにぎり
    1 昼食
    1 カクテル
    1 ジュース
    1 おにぎり
    1 勝谷菓子パン舗
    1 アナゴ飯
    1 アナゴ飯弁当
    1 地ビール
    1 宮島ビール
    1 作り立て
    1 アイス
    1 レモンジュース
    1 穴子
    1 抹茶
    1 クリーム
    1 カワウソ
    1 ビール
    1 殻付き牡蠣
    1 牡蠣のパスタ
    1 牡蠣入り
    1 牡蠣入りのカレーパン
    1 ごはん
    1 お茶
    1 焼き立て
    1 スターバックスコーヒー
    1 もみじ饅頭焼き
